In [ ]:
import folium
import requests
import osmnx as ox
import networkx as nx
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [ ]:
# Añadir marcadores al mapa
def add_marck_to_map(mapa, locaciones):
    # cada locación es una lista [latitud, longitud, nombre]
    for loc in locaciones:
        folium.Marker(
            location=[loc[0], loc[1]],
            popup=f"<b>{loc[2]}</b>",
            tooltip=loc[2] # Texto que aparece al pasar el cursor
        ).add_to(mapa)

In [ ]:
# Añadir rutas al mapa
def add_route_to_map(mapa, rutas):
    # Cada ruta es una lista de coordenadas [lat, lon]
    
    color = ['blue', 'green', 'red', 'purple', 'orange']
    
    for i, rut in enumerate(rutas):
        folium.PolyLine(
            locations=rut,
            color=color[i % len(color)],
            weight=5,
            opacity=0.7
        ).add_to(mapa)

In [ ]:
def get_route_for_streets(coord_inicio, coord_fin):
    """
    Calcula la ruta por carretera entre dos puntos usando la API de OSRM.
    Devuelve una lista de coordenadas [lat, lon].
    """
    # Formato de coordenadas para la URL de OSRM es {lon},{lat}
    lon_inicio, lat_inicio = coord_inicio[1], coord_inicio[0]
    lon_fin, lat_fin = coord_fin[1], coord_fin[0]

    url = f"http://router.project-osrm.org/route/v1/driving/{lon_inicio},{lat_inicio};{lon_fin},{lat_fin}?overview=full&geometries=geojson"

    try:
        respuesta = requests.get(url)
        datos_ruta = respuesta.json()
        # Extraer las coordenadas de la geometría de la ruta
        coordenadas_ruta = datos_ruta['routes'][0]['geometry']['coordinates']
        # OSRM devuelve [lon, lat], folium necesita [lat, lon], así que las invertimos
        puntos_ruta = [[lat, lon] for lon, lat in coordenadas_ruta]
        return puntos_ruta
    except Exception as e:
        print(f"No se pudo obtener la ruta entre {coord_inicio} y {coord_fin}: {e}")
        return []

In [ ]:
# Crear matriz de distancias
def crear_matriz_de_distancias(locaciones, grafo):
    nodos_locaciones = [ox.nearest_nodes(grafo, Y=loc[0], X=loc[1]) for loc in locaciones]
    n = len(locaciones)
    matriz = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i, n):
            longitud = nx.shortest_path_length(grafo, source=nodos_locaciones[i], target=nodos_locaciones[j], weight='length')
            matriz[i][j] = longitud
            matriz[j][i] = longitud
    return matriz

In [ ]:
# Resolver TSP con OR-Tools
def resolver_tsp_con_ortools(matriz_distancias):
    manager = pywrapcp.RoutingIndexManager(len(matriz_distancias), 1, 0)
    routing = pywrapcp.RoutingModel(manager)
    def distance_callback(from_index, to_index):
        return matriz_distancias[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        ruta_optima_indices = []
        index = routing.Start(0)
        while not routing.IsEnd(index):
            ruta_optima_indices.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        return ruta_optima_indices
    return None

In [ ]:
# Ordenar locaciones
def sort_locations(locaciones, matriz_distancias):
    """
    Ordena las locaciones usando el algoritmo TSP.
    Devuelve una lista de índices de las locaciones ordenadas.
    """
    orden_optimo_indices = resolver_tsp_con_ortools(matriz_distancias)
    
    if not orden_optimo_indices:
        print("No se pudo encontrar una ruta óptima.")
        return None
    
    locaciones_ordenadas = [locaciones[i] for i in orden_optimo_indices]
    
    return locaciones_ordenadas

In [ ]:
def calculate_routes(locaciones):
    """
    Calcula las rutas entre una lista de locaciones.
    Devuelve una lista de rutas, donde cada ruta es una lista de coordenadas [lat, lon].
    """
    
    if len(locaciones) < 2:
        print("Se necesitan al menos 2 locaciones para crear una ruta.")
        return []

    # 3. Reordenar locaciones y generar tramos de ruta
    # Creamos una lista de puntos y cerramos el ciclo añadiendo el primero al final
    puntos_ciclo = locaciones + [locaciones[0]]
    rutas = []
    
    print(f"Calculando {len(locaciones)} tramos de ruta...")

    for i in range(len(puntos_ciclo) - 1):
        punto_a = puntos_ciclo[i]
        punto_b = puntos_ciclo[i+1]

        # Extraemos solo las coordenadas [lat, lon]
        coord_a = [punto_a[0], punto_a[1]]
        coord_b = [punto_b[0], punto_b[1]]
        
        # Obtenemos la ruta para el tramo
        ruta_tramo = get_route_for_streets(coord_a, coord_b)
        
        if ruta_tramo:
            rutas.append(ruta_tramo)
            print(f"  > Ruta de '{punto_a[2]}' a '{punto_b[2]}' calculada.")
        else:
            print(f"  > Falló la ruta de '{punto_a[2]}' a '{punto_b[2]}'.")
            
    print("¡Proceso completado!")
    return rutas

In [ ]:
# Graficar matriz de distancias
def plot_distance_matrix(matriz_distancias, locaciones):
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set(style='whitegrid')

    # Crear un DataFrame para la matriz de distancias
    import pandas as pd
    df = pd.DataFrame(matriz_distancias, index=[loc[2] for loc in locaciones], columns=[loc[2] for loc in locaciones])

    # Graficar la matriz de distancias
    plt.figure(figsize=(10, 8))
    sns.heatmap(df, annot=True, fmt=".1f", cmap='coolwarm', square=True)
    plt.title('Matriz de Distancias')
    plt.show()

In [ ]:
# Ubicaciones en Tacna que usaremos como ejemplo
locaciones_tacna = [
    (-18.025947, -70.251223, 'Puerta de la U Av. Cusco'), # Punto inicial (Puerta de la U Av. Cusco)
    (-18.018047, -70.253128, 'Plaza Vea'), # Plaza Vea
    (-18.013997, -70.248859, 'DM Hotel'), # DM Hotel
    (-18.013682, -70.250590, 'Arco parabolico'), # Arco parabolico
    (-18.016574, -70.252357, 'Ferrocarril'), # Ferrocarril
    (-18.013078, -70.254401, 'Museo Ferrobiario'), # Museo Ferrobiario
]

In [ ]:
center_point = [-18.027808, -70.251069]

print("Descargando el grafo de calles de Tacna...")
G = ox.graph_from_point(center_point, dist=10000, network_type="drive")
fig,ax = ox.plot_graph(G, bgcolor="black", node_size=5 , node_color="white", figsize=(16,8))
print("Grafo local listo.")

In [ ]:
matriz_distancias = crear_matriz_de_distancias(locaciones_tacna, G)
plot_distance_matrix(matriz_distancias, locaciones_tacna)

In [ ]:
locaciones_tacna = sort_locations(locaciones_tacna, matriz_distancias)
print(f"Locaciones ordenadas por TSP: {locaciones_tacna}")

In [ ]:
# Calcular las rutas entre las locaciones
rutas_calculadas = calculate_routes(locaciones_tacna)

In [ ]:
# Crear el mapa base de Tacna
# Coordenadas del centro de Tacna
mapa_tacna = folium.Map(location=center_point, zoom_start=14)

# Agregar los marcadores y las rutas al mapa
add_marck_to_map(mapa_tacna, locaciones_tacna)
add_route_to_map(mapa_tacna, rutas_calculadas)

# Visualizar el mapa
mapa_tacna